# Backend Demo

In [1]:
from requests import get, post
from Database import Database

if 'db' not in locals():
    db = Database(
        {
            'user':              "root",
            'password':          "real_good_password",
            'database':          "backend",
            'host':              "localhost",
            'port':              "3306",
            'raise_on_warnings': True,
        }
    )

server_url = "http://127.0.0.1:105"
username = 'user2@domain.tld'

Creating table users: already exists.
Creating table games: already exists.
Creating table moves: already exists.


## Starting a game through the API

In [2]:
def start_with_white():
    response = post(
        server_url + "/start-with-white",
        headers={
            'Content-Type': 'application/json',
        },
        json={
            "username": username,
        },
    )
    print(response.status_code)
    return response.text

start_with_white()

200


'Ready to start'

In [3]:
def see_board():
    response = get(
        server_url + "/see-board",
    )
    print(response.status_code)
    print(response.text)

see_board()

200
+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
| p | p | p | p | p | p | p | p | 7
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
| P | P | P | P | P | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h


## Playing a move through the API

In [4]:
def play(move):
    currX = ord(move[0]) - 97  # convert character to unicode, then to index starting at 0.
    currY = int(move[1]) - 1  # backend expects moves indexed from 0.
    newX = ord(move[2]) - 97
    newY = int(move[3]) - 1 
    # print(currX, currY, newX, newY, sep="")
    
    response = post(
        server_url + "/play",
        headers={
            'Content-Type': 'application/json',
        },
        json={
            "username": username,
            "currX": currX,
            "currY": currY,
            "newX": newX,
            "newY": newY,
        },
    )
    print(response.status_code)
    print(response.text)

play("a2a4")

200
{"new_engine_x":0,"new_engine_y":5,"old_engine_x":0,"old_engine_y":6}


In [5]:
see_board()

200
+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
|   | p | p | p | p | p | p | p | 7
+---+---+---+---+---+---+---+---+
| p |   |   |   |   |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
| P |   |   |   |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
|   | P | P | P | P | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h


In [6]:
play("a2a4")

400
Move was illegal and was not made


In [7]:
play("d2d4")
see_board()

200
{"new_engine_x":5,"new_engine_y":5,"old_engine_x":6,"old_engine_y":7}

200
+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b |   | r | 8
+---+---+---+---+---+---+---+---+
|   | p | p | p | p | p | p | p | 7
+---+---+---+---+---+---+---+---+
| p |   |   |   |   | n |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
| P |   |   | P |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
|   | P | P |   | P | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h


## Getting moves from the `moves` Database tablet throught the API

In [8]:
def get_moves():
    response = get(
        server_url + "/get-moves",
        headers={
            'Content-Type': 'application/json',
        },
        json={
            "username": username,
        },
    )
    print(response.status_code)
    print(response.text)

get_moves()

200
["a2a4","a7a6","d2d4","g8f6"]


## Last Row of `users` and `games` tables from the Database

### User => (pk, email)

In [9]:
with db.get_cxn() as cxn:
    with cxn.cursor() as cursor:
        cursor.execute(
            "SELECT * FROM users"
        )
        result_set = cursor.fetchall()
        print(result_set[-1:])
        

[(2, 'user2@domain.tld')]


### Game => (pk, user_foreign_key, game_time)

In [10]:
with db.get_cxn() as cxn:
    with cxn.cursor() as cursor:
        cursor.execute(
            "SELECT * FROM games"
        )
        result_set = cursor.fetchall()
        print(result_set[-1:])

[(22, 2, datetime.datetime(2023, 12, 11, 20, 35, 38))]


<h2> 4 Most Recent Games

### Move => (pk, game_foreign_key, move_idx, move)

In [11]:
with db.get_cxn() as cxn:
    with cxn.cursor() as cursor:
        cursor.execute(
            "SELECT * FROM moves"
        )
        result_set = cursor.fetchall()
        print(result_set[-4:])

[(77, 22, 0, 'a2a4'), (78, 22, 1, 'a7a6'), (79, 22, 2, 'd2d4'), (80, 22, 3, 'g8f6')]


In [12]:
get_moves()

200
["a2a4","a7a6","d2d4","g8f6"]
